# Regular binning of NGVS data to VERTICO resolutions (either 9 arcsec beam with 2 arcsec pixels or Nyquist resolution)

Isaac Cheng - November 2021

Relative magnitudes are corrected for extinction.


In [1]:
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import reproject
from astropy.io import fits
from astropy.wcs import WCS
import dill
import pandas as pd
import reproject
from multiprocessing import Pool
from itertools import repeat
# 
# Load my own packages
# 
import sys
sys.path.append("/arc/home/IsaacCheng/coop_f2021/packages")
import fits_utils as fu
import plot_utils as pu
# # Reload my packages
# import importlib
# importlib.reload(fu)
# importlib.reload(pu)
GALDIST = 16.5 * u.Mpc  # distance to centre of Virgo cluster. Assume 10% uncertainty

In [ ]:
ngvs_catalogue = pd.read_csv("/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS_catalogue.txt", sep=" ", low_memory=False)
ngvs_catalogue

In [3]:
GALAXIES = [
    "IC3392",
    # "NGC4189",  # ignored flag map, but bad results
    "NGC4192",
    "NGC4216",  # has unflagged foreground star
    "NGC4222",
    "NGC4254",
    "NGC4294",
    "NGC4298",
    "NGC4299",
    "NGC4302",
    "NGC4321",
    "NGC4330",
    "NGC4351",
    "NGC4380",
    "NGC4383",
    "NGC4388",
    "NGC4396",
    "NGC4402",
    "NGC4405",
    "NGC4419",
    "NGC4424",
    "NGC4450",
    "NGC4501",
    "NGC4522",
    "NGC4532",
    "NGC4535",
    "NGC4548",
    "NGC4567",
    "NGC4568",
    "NGC4569",
    "NGC4579",
    "NGC4580",
    # "NGC4606",  # flag map too intrusive
    "NGC4607",
    "NGC4651",
    "NGC4654",
    "NGC4689",
    "NGC4694",
]
VCC_GALAXIES = [
    1126,  # IC3392
    # 89,  # NGC4189, ignored flag map, but bad results
    92,  # NGC4192
    167,  # NGC4216, has unflagged foreground star
    187,  # NGC4222
    307,  # NGC4254
    465,  # NGC4294
    483,  # NGC4298
    491,  # NGC4299
    497,  # NGC4302
    596,  # NGC4321
    630,  # NGC4330
    692,  # NGC4351
    792,  # NGC4380
    801,  # NGC4383
    836,  # NGC4388
    865,  # NGC4396
    873,  # NGC4402
    874,  # NGC4405
    958,  # NGC4419
    979,  # NGC4424
    1110,  # NGC4450
    1401,  # NGC4501
    1516,  # NGC4522
    1554,  # NGC4532
    1555,  # NGC4535
    1615,  # NGC4548
    1673,  # NGC4567
    1676,  # NGC4568
    1690,  # NGC4569
    1727,  # NGC4579
    1730,  # NGC4580
    # 1859,  # NGC4606, flag map too intrusive
    1868,  # NGC4607
    -100,  # NGC4651 (EVCC number is 1102, cannot use EVCC number)
    1987,  # NGC4654
    2058,  # NGC4689
    2066,  # NGC4694
]

ngvs_ra = ngvs_catalogue["NGVS_ra(deg)"]
ngvs_dec = ngvs_catalogue["NGVS_dec(deg)"]
vcc_name = ngvs_catalogue["VCC_name"]
extinctions = ngvs_catalogue["e(B-V)"]

# Single entry that is closest to NGC4651 coordinates
NGC4651_row = vcc_name[
    (abs(ngvs_ra - 190.927625) < 0.1) & (abs(ngvs_dec - 16.39338889) < 0.1)
].index.values[0]

# E(B-V) to u, g, i, z extinction coefficients
# (from http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/query/catdoc.html#ebv)
EBV_u = 4.594
EBV_g = 3.560
EBV_i = 1.813
EBV_z = 1.221

# Regular binning procedure

1. Cut out the optical data to the radio data's extent. Remember to modify the WCS object.
2. Reproject the optical image (while preserving flux) so that its dimensions are integer
   multiples of the radio data's dimensions. Furthermore, the new dimensions should
   satisfy the above criterion while being as close as possible to the cutout's original
   dimensions. (This is because reprojection tries to preserve the pixel values as much as
   possible, regardless of the new dimensions.) Remember to modify the WCS object
   (tricky!).
3. Bin the reprojected optical image to the same resolution as the radio data. Remember to
   modify the WCS object (which should now be identical to the radio data's WCS object).
   

In [4]:
def bin_ngvs_data(
    galaxy,
    vcc_galaxy,
    bin_resolution,
    reproject_func=reproject.reproject_interp,
    bin_func=np.nansum,
    print_debug=False,
    plot_debug=False,
):
    # 
    # Check inputs
    # 
    if bin_resolution != "nyquist" and bin_resolution != "9as":
        raise ValueError("bin_resolution must be 'nyquist' or '9as'")
    if reproject_func == reproject.reproject_interp:
        reproject_str = "reproject_interp"
    elif reproject_func == reproject.reproject_exact:
        reproject_str = "reproject_exact"
    else:
        raise ValueError(
            "reproject_func must be either reproject_interp or reproject_exact"
        )
    if bin_func == np.nansum:
        bin_func_str = "nansum"
    elif bin_func == np.sum:
        bin_func_str = "sum"
    else:
        raise ValueError("bin_func must be either np.nansum or np.sum")
    # 
    # Get NGVS filepaths
    #
    if print_debug:
        print(f"On {galaxy} (VCC {vcc_galaxy}) at {bin_resolution} resolution")
    galpath = f"/arc/home/IsaacCheng/coop_f2021/galaxies/{galaxy}/"
    # Signal
    zband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_z_data.fits"
    iband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_i_data.fits"
    gband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_g_data.fits"
    uband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_u_data.fits"
    # Noise (sigma maps)
    znoise_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_z_sig.fits"
    inoise_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_i_sig.fits"
    gnoise_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_g_sig.fits"
    unoise_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_u_sig.fits"
    # Mask (flag maps)
    zgood_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_z_flag.fits"
    igood_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_z_flag.fits"
    ggood_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_z_flag.fits"
    ugood_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_z_flag.fits"
    # 
    # Load NGVS data
    # 
    # Signal
    zband_data_full, zband_header_full = fits.getdata(zband_path, header=True)
    iband_data_full, iband_header_full = fits.getdata(iband_path, header=True)
    gband_data_full, gband_header_full = fits.getdata(gband_path, header=True)
    uband_data_full, uband_header_full = fits.getdata(uband_path, header=True)
    # Noise
    zband_noise_full = fits.getdata(znoise_path)
    iband_noise_full = fits.getdata(inoise_path)
    gband_noise_full = fits.getdata(gnoise_path)
    uband_noise_full = fits.getdata(unoise_path)
    # Masks
    zband_good_full = fits.getdata(zgood_path, ext=1)
    iband_good_full = fits.getdata(igood_path, ext=1)
    gband_good_full = fits.getdata(ggood_path, ext=1)
    uband_good_full = fits.getdata(ugood_path, ext=1)
    #
    # Mask the data and noise arrays
    #
    uband_data_masked = np.ma.masked_array(uband_data_full, mask=uband_good_full == 0).filled(fill_value=np.nan)
    uband_noise_masked = np.ma.masked_array(uband_noise_full, mask=uband_good_full == 0).filled(fill_value=np.nan)
    gband_data_masked = np.ma.masked_array(gband_data_full, mask=gband_good_full == 0).filled(fill_value=np.nan)
    gband_noise_masked = np.ma.masked_array(gband_noise_full, mask=gband_good_full == 0).filled(fill_value=np.nan)
    iband_data_masked = np.ma.masked_array(iband_data_full, mask=iband_good_full == 0).filled(fill_value=np.nan)
    iband_noise_masked = np.ma.masked_array(iband_noise_full, mask=iband_good_full == 0).filled(fill_value=np.nan)
    zband_data_masked = np.ma.masked_array(zband_data_full, mask=zband_good_full == 0).filled(fill_value=np.nan)
    zband_noise_masked = np.ma.masked_array(zband_noise_full, mask=zband_good_full == 0).filled(fill_value=np.nan)
    # 
    # VERTICO CO Data
    # 
    if bin_resolution == "nyquist":
        if galaxy == "NGC4321":  # no 9 arcsec data since native is 10 arcsec. Use 15 arcsec data instead
            Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3.1/nyquistPix/15arcsec/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_15as_np_round_mom0_Kkms-1.fits"
        else:
            Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/nyquistPix/9arcsec/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_9as_np_round_mom0_Kkms-1.fits"
    else:
        if galaxy == "NGC4321":  # no 9 arcsec data since native is 10 arcsec
            Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3.1/native/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_round_mom0_Kkms-1.fits"
        else:
            Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/9arcsec/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_9as_round_mom0_Kkms-1.fits"
    Ico_data, co_header = fits.getdata(Ico_path, header=True)
    co_wcs = WCS(co_header)
    # 
    # Bin NGVS data to VERTICO resolution
    # 
    (
        xs_binned,
        ys_binned,
        uband_data_binned,
        uband_noise_binned,
        uband_isgood_binned,
        wcs_binned,
        bin_dimensions,
    ) = fu.bin_sn_arrs_to_target(
        uband_data_masked,
        WCS(uband_header_full),
        uband_noise_masked,
        WCS(uband_header_full),
        Ico_data,
        co_wcs,
        reproject_func=reproject_func,
        bin_func=np.nansum,
        print_debug=print_debug,
        return_bin_dimen=True,
    )
    (
        _,
        _,
        gband_data_binned,
        gband_noise_binned,
        gband_isgood_binned,
        _,
        tmpgdim,
    ) = fu.bin_sn_arrs_to_target(
        gband_data_masked,
        WCS(gband_header_full),
        gband_noise_masked,
        WCS(gband_header_full),
        Ico_data,
        co_wcs,
        reproject_func=reproject_func,
        bin_func=np.nansum,
        print_debug=False,
        return_bin_dimen=True,
    )
    (
        _,
        _,
        iband_data_binned,
        iband_noise_binned,
        iband_isgood_binned,
        _,
        tmpidim,
    ) = fu.bin_sn_arrs_to_target(
        iband_data_masked,
        WCS(iband_header_full),
        iband_noise_masked,
        WCS(iband_header_full),
        Ico_data,
        co_wcs,
        reproject_func=reproject_func,
        bin_func=np.nansum,
        print_debug=False,
        return_bin_dimen=True,
    )
    (
        _,
        _,
        zband_data_binned,
        zband_noise_binned,
        zband_isgood_binned,
        _,
        tmpzdim,
    ) = fu.bin_sn_arrs_to_target(
        zband_data_masked,
        WCS(zband_header_full),
        zband_noise_masked,
        WCS(zband_header_full),
        Ico_data,
        co_wcs,
        reproject_func=reproject_func,
        bin_func=np.nansum,
        print_debug=False,
        return_bin_dimen=True,
    )
    # 
    # Check all bin dimensions are equal
    # 
    if (
        (not np.all(bin_dimensions == tmpgdim))
        or (not np.all(bin_dimensions == tmpidim))
        or (not np.all(bin_dimensions == tmpzdim))
    ):
        print(bin_dimensions, tmpgdim, tmpidim, tmpzdim)
        raise ValueError("Bin dimensions are not equal")
    # 
    # Normalize fluxes by the number of pixels in each bin
    #
    if plot_debug:
        fig, ax = plt.subplots(subplot_kw={"projection": wcs_binned})
        img = ax.imshow(uband_data_binned, cmap="plasma", norm=mpl.colors.LogNorm())
        cbar = fig.colorbar(img)
        cbar.ax.tick_params(which="both", direction="out")
        ax.contour(
            Ico_data,
            transform=ax.get_transform(co_wcs),
            levels=[0, 5, 10],
            colors="w",
        )
        ax.set_title("Un-normalized u-band flux")
        ax.set_xlabel("RA (J2000)")
        ax.set_ylabel("Dec (J2000)")
        ax.grid(False)
        ax.set_aspect("equal")
        plt.show()
    px_per_bin = bin_dimensions[0] * bin_dimensions[1]
    uband_data_binned = uband_data_binned / px_per_bin
    uband_noise_binned = uband_noise_binned / px_per_bin
    gband_data_binned = gband_data_binned / px_per_bin
    gband_noise_binned = gband_noise_binned / px_per_bin
    iband_data_binned = iband_data_binned / px_per_bin
    iband_noise_binned = iband_noise_binned / px_per_bin
    zband_data_binned = zband_data_binned / px_per_bin
    zband_noise_binned = zband_noise_binned / px_per_bin
    if plot_debug:
        fig, ax = plt.subplots(subplot_kw={"projection": wcs_binned})
        img = ax.imshow(uband_data_binned, cmap="plasma", norm=mpl.colors.LogNorm())
        cbar = fig.colorbar(img)
        cbar.ax.tick_params(which="both", direction="out")
        ax.contour(
            Ico_data,
            transform=ax.get_transform(co_wcs),
            levels=[0, 5, 10],
            colors="w",
        )
        ax.set_title("Normalized u-band flux")
        ax.set_xlabel("RA (J2000)")
        ax.set_ylabel("Dec (J2000)")
        ax.grid(False)
        ax.set_aspect("equal")
        plt.show()
    # 
    # Calculate relative AB magnitudes
    # 
    uband_rel_mag_binned, uband_rel_mag_err_binned = fu.calc_mag(
        uband_data_binned, uband_noise_binned, zpt=30.0, calc_abs=False
    )
    gband_rel_mag_binned, gband_rel_mag_err_binned = fu.calc_mag(
        gband_data_binned, gband_noise_binned, zpt=30.0, calc_abs=False
    )
    iband_rel_mag_binned, iband_rel_mag_err_binned = fu.calc_mag(
        iband_data_binned, iband_noise_binned, zpt=30.0, calc_abs=False
    )
    zband_rel_mag_binned, zband_rel_mag_err_binned = fu.calc_mag(
        zband_data_binned, zband_noise_binned, zpt=30.0, calc_abs=False
    )
    # 
    # Correct for extinction
    # 
    if galaxy != "NGC4651":
        ebv_coeff = extinctions[vcc_name == vcc_galaxy].values
        if ebv_coeff.size != 1:
            raise ValueError(f"More than one E(B-V) coefficient found for {galaxy}")
        ebv_coeff = ebv_coeff[0]
    else:
        ebv_coeff = extinctions.iloc[NGC4651_row]
    uband_rel_mag_binned = uband_rel_mag_binned - ebv_coeff * EBV_u
    gband_rel_mag_binned = gband_rel_mag_binned - ebv_coeff * EBV_g
    iband_rel_mag_binned = iband_rel_mag_binned - ebv_coeff * EBV_i
    zband_rel_mag_binned = zband_rel_mag_binned - ebv_coeff * EBV_z
    # 
    # Pickle data
    # 
    galaxy_outfile = galpath + f"{galaxy}_regBin_extinctionCorr_{bin_resolution}.pkl"
    with open(galaxy_outfile, "wb") as f:
        dill.dump(
            {
                "README": "The binned signal and noise arrays, as well as the relative magnitudes, have all been normalized by the number of pixels per bin. "
                + "To recover the un-normalized binned signal and noise arrays, simply multiply the signal and noise arrays by (`bin_dimensions`[0] * `bin_dimensions`[1]). "
                + "\nThe relative magnitudes have been corrected for extinction; the binned signal and noise arrays have not. "
                + "\n`ebv_coeff` is the extinction coefficient for the galaxy from the NGVS catalogue; "
                + "`ebv_u`, `ebv_g`, `ebv_i`, `ebv_z` are the NGVS extinction coefficient conversion factors from http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/query/catdoc.html#ebv. "
                + "\nAlso remember to set `wcs_binned.array_shape = wcs_binned_array_shape`",
                
                "reproject_method": reproject_str,
                "binning_function": bin_func_str,
                "dist_pc": GALDIST.to(u.pc).value,  # distance in parsecs
                "x_coords": xs_binned,  # x-value of pixel coordinates
                "y_coords": ys_binned,  # y-value of pixel coordinates
                "bin_dimensions": bin_dimensions,
                "wcs_binned": wcs_binned,  # dill has trouble saving the "NAXIS" keyword
                "wcs_binned_array_shape": wcs_binned.array_shape,  # "NAXIS" keyword
                "ebv_coeff": ebv_coeff,  # extinction coefficient for the galaxy
                "ebv_u": EBV_u,  # extinction coefficient for the galaxy
                
                "uband_signal": uband_data_binned,
                "uband_noise": uband_noise_binned,
                "gband_signal": gband_data_binned,
                "gband_noise": gband_noise_binned,
                "iband_signal": iband_data_binned,
                "iband_noise": iband_noise_binned,
                "zband_signal": zband_data_binned,
                "zband_noise": zband_noise_binned,
                
                "uband_rel_mag": uband_rel_mag_binned,
                "uband_rel_mag_err": uband_rel_mag_err_binned,
                "gband_rel_mag": gband_rel_mag_binned,
                "gband_rel_mag_err": gband_rel_mag_err_binned,
                "iband_rel_mag": iband_rel_mag_binned,
                "iband_rel_mag_err": iband_rel_mag_err_binned,
                "zband_rel_mag": zband_rel_mag_binned,
                "zband_rel_mag_err": zband_rel_mag_err_binned,
                
                "uband_is_good": uband_isgood_binned,
                "gband_is_good": gband_isgood_binned,
                "iband_is_good": iband_isgood_binned,
                "zband_is_good": zband_isgood_binned,
            },
            f,
        )
        print(f"Pickled {galaxy_outfile}")


In [ ]:
# # 
# # Preliminary testing and sanity checks
# # 
# bin_ngvs_data("IC3392", 1126, bin_resolution="nyquist", plot_debug=True, reproject_func=reproject.reproject_interp, bin_func=np.nansum)
# print()
# with open("/arc/home/IsaacCheng/coop_f2021/galaxies/IC3392/IC3392_regBin_extinctionCorr_nyquist.pkl", "rb") as f:
#     file = dill.load(f)
#     print(file.keys())
#     print()
#     print(file["README"])
#     print()
#     print(file["reproject_method"])
#     print(file["binning_function"])
#     print(file["wcs_binned"])
#     print(file["wcs_binned"])
#     print(file["wcs_binned_array_shape"])
#     print(file["ebv_coeff"])
#     print(file["ebv_u"])
#     print(file["uband_signal"].shape)
    
# tmppath = "/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/nyquistPix/9arcsec/IC3392/IC3392_7m+tp_co21_pbcorr_9as_np_round_mom0_Kkms-1.fits"
# tmpdata, tmpheader = fits.getdata(tmppath, header=True)
# print("-" * 10)
# print(tmpdata.shape)
# print(WCS(tmpheader))

## Nyquist (4 arcsec pixel) Resolution


In [ ]:
# for gal, vcc_gal in zip(GALAXIES, VCC_GALAXIES):
#     bin_ngvs_data(
#         gal,
#         vcc_gal,
#         "nyquist",
#         reproject_func=reproject.reproject_interp,
#         bin_func=np.nansum,
#     )
# print("Done!")

In [ ]:
with Pool(processes=None) as p:
    p.starmap(bin_ngvs_data, zip(GALAXIES, VCC_GALAXIES, repeat("nyquist")))
print("Done")

## 9 arcsec beam with 2 arcsec pixels Resolution


In [ ]:
with Pool(processes=None) as p:
    p.starmap(bin_ngvs_data, zip(GALAXIES, VCC_GALAXIES, repeat("9as")))
print("Done")